# 🦜🔗 LangChain zoomcamp - Session 1!



In [ ]:
%%capture
!pip install langchain openai gradio tiktoken faiss-cpu

In [ ]:
import os
import re
import getpass
import langchain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from typing import List, Union

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# Building a Context-Aware Chatbot using Web Data and OpenAI

In the vast ocean of information available on the web, extracting relevant insights can sometimes be a daunting task.

But what if you could have a chatbot that can crawl websites, understand their content, and provide you with concise answers?

In our latest project, we've built a chatbot that does just that!

Let's walk you through the steps:

# 1. Extracting URLs from Text

First, we identify URLs from a given text.

This can be any text containing references to websites that you want the chatbot to understand.

```
url_lists = find_urls(text)
```

# 2. Load the documents from the web using the WebBaseLoader.

Once we have the list of URLs, we load these websites into a document structure, which makes it easier for further processing.

This loader allows you to load documents from various sources such as public websites.

You can specify the URL of the website or webpage you want to load.

```
documents = website_loader(url_lists)
```

# 3. Transform the loaded documents using document transformers.

Large websites can have overwhelming amounts of content.

To make it manageable, we split the content into smaller chunks.

This step involves preparing the documents for retrieval by applying transformations such as splitting or chunking large documents into smaller chunks. LangChain provides different algorithms for document transformation, optimized for specific document types.

```
chunks = split_text(documents)
```

# 4. Create text embeddings for the transformed documents.

For our chatbot to understand the content, we convert these chunks into vector representations using an embedding model from OpenAI.

Text embeddings capture the semantic meaning of the text and enable efficient searching for similar pieces of text.

LangChain offers integrations with various text embedding models, allowing you to choose the one that best suits your needs.

There's a couple of ways you can split text, using either the  `CharacterTextSplitter` or the `RecursiveTextSplitter`, the difference lies in how they split the text and measure the chunk size.

`CharacterTextSplitter` splits the text based on characters passed in and measures the chunk size by the number of characters.

On the other hand, `RecursiveTextSplitter` splits the text into smaller chunks based on semantically meaningful units (such as sentences) and measures the chunk size using a length function (which can be the number of characters or a token counter).

To determine which splitter to use, you can consider the following:

 - If you want to split the text based on characters and measure the chunk size by the number of characters, you can use CharacterTextSplitter.

 - If you want to split the text into smaller chunks based on semantically meaningful units and measure the chunk size using a length function (such as the number of characters or a token counter), you can use RecursiveTextSplitter.

```
embedder = get_document_embeddings(chunks)
```

# 5. Store the embeddings in a vector store.

Once embedded, these vectors are stored in a FAISS vector store, a highly efficient structure for similarity searches.

Vector stores are databases designed to efficiently store and search embeddings.

LangChain provides integrations with different vector stores, both open-source and cloud-hosted proprietary ones.

You can select the vector store that suits your requirements.

```
vectorstore = create_vector_store(chunks, embedder)
```

## 6. Use a retriever to retrieve the relevant documents.

To retrieve relevant chunks based on user queries, we create a retriever from the vector store.

LangChain supports various retrieval algorithms, including simple semantic search and advanced methods like the Parent Document Retriever, Self Query Retriever, and Ensemble Retriever.

These retrievers help you retrieve the documents based on the user's query.

```
retriever = create_retriever(vectorstore)
```

# 7. Building the Chatbot

Finally, we assemble our chatbot using the retriever and OpenAI's model. This chatbot understands the context from the web data and can provide relevant answers.

We use the `ConversationalRetrievalChain`for having a conversation based on retrieved documents.

This chain takes in chat history (a list of messages) and new questions,
and then returns an answer to that question.

The algorithm for this chain consists of three parts:

1. Use the chat history and the new question to create a "standalone question".

  This is done so that this question can be passed into the retrieval step to fetch relevant documents.

  If only the new question was passed in, then relevant context
  may be lacking.

  If the whole conversation was passed into retrieval, there may
  be unnecessary information there that would distract from retrieval.

2. This new question is passed to the retriever and relevant documents are
returned.

3. The retrieved documents are passed to an LLM along with either the new question (default behavior) or the original question and chat history to generate a final response.


```
chatbot = create_chatbot(retriever)
```

# Testing the Chatbot

Now, it's time for the fun part! We can interact with the chatbot:

```
response = chat("How do I not waste time with my projects")
print(response)
```

In [ ]:
def find_urls(text: str) -> List:
    """
    Extract URLs from a given text.

    This function looks for patterns starting with 'http://', 'https://', or 'www.'
    followed by any non-whitespace characters. It captures common URL formats
    but might not capture all possible URL variations.

    Args:
    - text (str): The input string from which URLs need to be extracted.

    Returns:
    - list: A list containing all the URLs found in the input text.
    """
    # Regular expression to match common URLs and ones starting with 'www.'
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.findall(text)

def website_loader(website: Union[str, list[str]]) -> List[langchain.schema.document.Document]:
    """
    Loads the specified website(s) into Document objects.

    This function initiates the WebBaseLoader with the provided website or list of websites,
    loads them, and returns the resulting Document objects.

    Parameters:
    - website (Union[str, list[str]]): A single website URL as a string or a list of website URLs to be loaded.

    Returns:
    - List[langchain.schema.document.Document]: A list of Document objects corresponding to the loaded website(s).
    """

    print("Loading website(s) into Documents...")
    documents = WebBaseLoader(web_path=website).load()
    print("Done loading website(s).")
    return documents

def split_text(documents: List) -> List[langchain.schema.document.Document]:
    """
    Splits the provided documents into chunks using RecursiveCharacterTextSplitter.

    This function takes a list of documents, splits each document into smaller chunks
    of a specified size with a specified overlap, and returns the chunks as a list of
    Document objects.

    Parameters:
    - documents (List): A list of Document objects to be split into chunks.

    Returns:
    - List[langchain.schema.document.Document]: A list of Document objects representing the chunks.

    Note:
    - The chunk size, overlap, and length function are set to 1000, 50, and len respectively. Adjust
      these values if necessary.
    """
    print("Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                   chunk_overlap=50,
                                                   length_function=len
                                                   )
    chunks = text_splitter.transform_documents(documents)
    print("Done splitting documents.")
    return chunks

def get_document_embeddings(chunks: List) -> langchain.embeddings.cache.CacheBackedEmbeddings:
    """
    Generates and retrieves embeddings for the given document chunks using CacheBackedEmbeddings.

    This function initializes an embedder backed by a local cache and a core embeddings model
    from OpenAI. It then uses this embedder to generate embeddings for the given document chunks.

    Parameters:
    - chunks (List): A list of Document chunks for which embeddings are to be generated.

    Returns:
    - langchain.embeddings.cache.CacheBackedEmbeddings: An embedder which can be used to get
      embeddings for the document chunks.
    """
    print("Creating embedder...")
    store = LocalFileStore("./cache/")
    core_embeddings_model= OpenAIEmbeddings()
    embedder = CacheBackedEmbeddings.from_bytes_store(
        core_embeddings_model,
        store,
        namespace=core_embeddings_model.model
    )
    print("Done creating embedder")
    return embedder

def create_vector_store(chunks: List[langchain.schema.document.Document],
                        embedder: langchain.embeddings.cache.CacheBackedEmbeddings) -> langchain.vectorstores.faiss.FAISS:
    """
    Creates a FAISS vector store from the given document chunks using the provided embedder.

    This function uses the provided embedder to transform the document chunks into vectors
    and then stores them in a FAISS vector store.

    Parameters:
    - chunks (List[langchain.schema.document.Document]): A list of Document chunks to be vectorized.
    - embedder (langchain.embeddings.cache.CacheBackedEmbeddings): An embedder used to generate embeddings
      for the document chunks.

    Returns:
    - langchain.vectorstores.faiss.FAISS: A FAISS vector store containing the vectors of the document chunks.
    """
    print("Creating vectorstore...")
    vectorstore = FAISS.from_documents(chunks, embedder)
    return vectorstore

def create_retriever(vectorstore: langchain.vectorstores) -> langchain.vectorstores.base.VectorStoreRetriever:
    """
    Creates a retriever for the provided FAISS vector store.

    This function initializes a retriever for the given vector store, allowing for efficient
    querying and retrieval of similar vectors/documents from the vector store.

    Parameters:
    - vectorstore (langchain.vectorstores): A FAISS vector store containing vectors of document chunks.

    Returns:
    - langchain.vectorstores.base.VectorStoreRetriever: A retriever object that can be used to query
      and retrieve similar vectors/documents from the vector store.

    """
    print("Creating vectorstore retriever...")
    retriever = vectorstore.as_retriever()
    return retriever

def embed_user_query(query: str) -> List[float]:
    """
    Embeds the provided user query using the OpenAIEmbeddings model.

    This function takes a user query as input and transforms it into a vector representation
    using the OpenAIEmbeddings model.

    Parameters:
    - query (str): The user query to be embedded.

    Returns:
    - List[float]: A list of floats representing the embedded vector of the user query.
    """
    core_embeddings_model = OpenAIEmbeddings()
    embedded_query = core_embeddings_model.embed_query(query)
    return embedded_query

def similarity_search(vectorstore: langchain.vectorstores,
                      embedded_query: List[float]) -> List[langchain.schema.document.Document]:
    """
    Performs a similarity search on the provided FAISS vector store using an embedded query.

    This function takes an embedded query and searches the FAISS vector store for the most
    similar vectors/documents based on the embedded query.

    Parameters:
    - vectorstore (langchain.vectorstores): A FAISS vector store containing vectors of document chunks.
    - embedded_query (List[float]): A list of floats representing the embedded vector of the user query.

    Returns:
    - List[langchain.schema.document.Document]: A list of Document objects that are the most similar to
      the embedded query.

    Note:
    - The function currently retrieves the top 4 most similar documents (k=4). Adjust the value of 'k'
      if a different number of results is desired.
    """
    response = vectorstore.similarity_search_by_vector(embedded_query, k=4)
    return response


def create_chatbot(retriever: langchain.vectorstores) -> langchain.chains.conversational_retrieval:
    """
    Initializes and returns a conversational chatbot using the provided retriever and the OpenAI model.

    This function sets up a chatbot based on the ConversationalRetrievalChain from LangChain,
    which leverages the OpenAI model for conversational interactions and uses the given retriever
    for document retrieval.

    Parameters:
    - retriever (langchain.vectorstores): A retriever object used for document retrieval based on similarity searches.

    Returns:
    - langchain.chains.conversational_retrieval: A ConversationalRetrievalChain instance which acts as the chatbot.

    Note:

    - The conversation history is stored in the 'chat_history' memory key and is used for context in
      subsequent interactions.
    """
    llm = ChatOpenAI(model="gpt-3.5-turbo")

    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
        )

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
        )
    return conversation_chain

def chat(conversation_chain, input: str) -> str:
    """
    Interacts with the chatbot using the provided input and returns its response.

    This function takes a user input, passes it to the chatbot for processing,
    and retrieves the chatbot's response.

    Parameters:
    - input (str): The user's input/question to the chatbot.

    Returns:
    - str: The chatbot's response to the user's input.

    """
    return conversation_chain.run(input)

text = """
I need you to go to the following URLs and get information about them
https://medium.com/cometheartbeat/how-to-prevent-yourself-from-wasting-time-on-your-data-science-project-728db69a4afc
and this one https://medium.com/mlearning-ai/langchain-is-the-past-here-is-the-future-of-llm-based-apps-46663f532c19
"""


In [ ]:
import gradio as gr

# This chatbot_instance will be initialized once a URL is provided.
chatbot_instance = None

def respond(message, chat_history):
    global chatbot_instance
    urls = find_urls(message)
    # If the chatbot is not yet initialized and we have URLs, initialize it
    if not chatbot_instance and urls:
        documents = website_loader(urls)
        chunks = split_text(documents)
        embedder = get_document_embeddings(chunks)
        vectorstore = create_vector_store(chunks, embedder)
        retriever = create_retriever(vectorstore)
        chatbot_instance = create_chatbot(retriever)
        bot_message = "Chatbot initialized! How can I help you?"
    else:
        if chatbot_instance:
            bot_message = chat(chatbot_instance, message)
        else:
            bot_message = "Please provide a URL to initialize the chatbot first."

    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    user_query = gr.Textbox(label="Your Query", placeholder="What would you like to chat about?")
    clear = gr.ClearButton([user_query, chatbot])

    user_query.submit(respond, [user_query, chatbot], [user_query, chatbot])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3f539f1fc49db6c261.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Loading website(s) into Documents...
Done loading website(s).
Splitting documents into chunks...
Done splitting documents.
Creating embedder...
Done creating embedder
Creating vectorstore...
Creating vectorstore retriever...
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://3f539f1fc49db6c261.gradio.live
